In [1]:
import pandas as pd
import mysql.connector
import csv

In [2]:
demand = pd.read_csv('demand_forecasting.csv')
inventory = pd.read_csv('inventory_monitoring.csv')
pricing = pd.read_csv('pricing_optimization.csv')
manager = pd.read_csv('manager_details.csv')
product = pd.read_csv('product_master.csv')

In [3]:
print("\nDemand")
demand.info()

print("\nInventory")
inventory.info()

print("\nPricing")
pricing.info()

print("\nManager")
manager.info()

print("\nProduct")
product.info()



Demand
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ProductID           10000 non-null  int64  
 1   Date                10000 non-null  object 
 2   StoreID             10000 non-null  int64  
 3   SalesQuantity       10000 non-null  int64  
 4   Price               10000 non-null  float64
 5   Promotions          10000 non-null  object 
 6   SeasonalityFactors  10000 non-null  object 
 7   DemandTrend         10000 non-null  object 
 8   CustomerSegments    10000 non-null  object 
 9   StoreLocation       10000 non-null  object 
 10  ManagerID           10000 non-null  object 
dtypes: float64(1), int64(3), object(7)
memory usage: 859.5+ KB

Inventory
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype
---  ------     

In [4]:
# Check for missing values in each DataFrame
print("Missing values before cleaning:")
print(f"\nDemand:\n{demand.isnull().sum()}\n")
print(f"Inventory:\n{inventory.isnull().sum()}\n")
print(f"Pricing:\n{pricing.isnull().sum()}\n")
print(f"Manager:\n{manager.isnull().sum()}\n")
print(f"Category:\n{product.isnull().sum()}\n")

Missing values before cleaning:

Demand:
ProductID             0
Date                  0
StoreID               0
SalesQuantity         0
Price                 0
Promotions            0
SeasonalityFactors    0
DemandTrend           0
CustomerSegments      0
StoreLocation         0
ManagerID             0
dtype: int64

Inventory:
ProductID               0
StoreID                 0
StockLevels             0
SupplierLeadTime        0
StockoutFrequency       0
ReorderPoint            0
WarehouseCapacity       0
OrderFulfillmentTime    0
dtype: int64

Pricing:
ProductID           0
StoreID             0
Price               0
CompetitorPrices    0
Discounts           0
SalesVolume         0
CustomerReviews     0
ReturnRate          0
StorageCost         0
dtype: int64

Manager:
ManagerID        0
ManagerName      0
StoreLocation    0
dtype: int64

Category:
ProductID      0
Category       0
SubCategory    0
dtype: int64



In [5]:
# Check and remove duplicate rows
print("\nDuplicate rows before cleaning:")
print(f"\nDemand: {demand.duplicated().sum()}")
print(f"Inventory: {inventory.duplicated().sum()}")
print(f"Pricing: {pricing.duplicated().sum()}")
print(f"Manager: {manager.duplicated().sum()}")
print(f"Category: {product.duplicated().sum()}")



Duplicate rows before cleaning:

Demand: 0
Inventory: 0
Pricing: 0
Manager: 0
Category: 0


In [ ]:

# Database connection configuration
db_config = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': 'mesh@2944',
    'database': 'meshva'
}

# List of CSV file paths and corresponding table names
csv_files = ['demand_forecasting.csv', 'inventory_monitoring.csv', 'pricing_optimization.csv', 'product_master.csv', 'manager_details.csv']
table_names = ['demand', 'inventory', 'pricing', 'Product', 'manager'] 

try:
    mydb = mysql.connector.connect(**db_config)
    mycursor = mydb.cursor()
    print("Successfully connected to MySQL database!")

    for i in range(len(csv_files)):
        csv_file_path = csv_files[i]
        table_name = table_names[i]

        try:
            with open(csv_file_path, 'r') as file:
                csv_reader = csv.reader(file)
                header = next(csv_reader) # Skip the header row

                placeholders = ', '.join(['%s'] * len(header))
                sql = f"INSERT INTO {table_name} ({', '.join(header)}) VALUES ({placeholders})"

                for row in csv_reader:
                    mycursor.execute(sql, row)

            mydb.commit()
            print(f"Data from '{csv_file_path}' loaded successfully into '{table_name}'.")

        except FileNotFoundError:
            print(f"Error: CSV file '{csv_file_path}' not found.")
        except mysql.connector.Error as err:
            print(f"Error loading data into '{table_name}' from '{csv_file_path}': {err}")
            mydb.rollback()
        except Exception as e:
            print(f"An unexpected error occurred while processing '{csv_file_path}': {e}")
            mydb.rollback()

except mysql.connector.Error as err:
    print(f"Error connecting to MySQL: {err}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    if 'mydb' in locals() and mydb.is_connected():
        mycursor.close()
        mydb.close()
        print("MySQL connection closed.")